In [1]:
import pandas as pd
import numpy as np
from hyperopt import fmin, tpe, hp
import ipynb.fs.full.utils as utils
import ipynb.fs.full.features as features
import ipynb.fs.full.features_distancias as f_distancias


df_train = pd.read_csv('./data/train_filtrado.csv')
# Para usarse con el submit a Kaggle
df_eval = pd.read_csv('./data/test.csv')

df_train, df_eval = features.features_de_csvs(df_train, df_eval)

df_train_idf = pd.read_csv('./data/train_idf.csv')
df_eval_idf = pd.read_csv('./data/test_idf.csv')

df_train = pd.merge(df_train, df_train_idf, on= 'id', how= 'left')
df_eval = pd.merge(df_eval, df_eval_idf, on= 'id', how= 'left')

df_train, df_test = utils.dividir_df_testeo(df_train, test_size=0.25)

df_test = features.llenar_nulls(df_test, hgb_mean=True, df_fill=df_train)
df_train = features.llenar_nulls(df_train, hgb_mean=True)

In [2]:
# df_train = df_train.sample(frac=1).reset_index(drop=True)

df_test_f = features.features_independientes_precio(df_test)
df_test_f = features.features_dependientes_precio(df_test_f, df_train)

df_train_f = features.features_independientes_precio(df_train)
df_train_f = features.features_dependientes_precio(df_train_f, df_train)

df_test_f, cols_tipodepropiedad_ohe = features.columna_a_ohe(df_test_f, 'tipodepropiedad', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_provincia_ohe = features.columna_a_ohe(df_test_f, 'provincia', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_zona_ohe = features.columna_a_ohe(df_test_f, 'zona', df_aux=df_train_f, devolver_cols=True)

df_train_f = features.columna_a_ohe(df_train_f, 'tipodepropiedad', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'provincia', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'zona', df_aux=df_test_f)


df_train_f['fecha'] = pd.to_datetime(df_train_f['fecha']).astype(int)
df_test_f['fecha'] = pd.to_datetime(df_test_f['fecha']).astype(int)

# df_train_f = df_train_f.sample(frac=1).reset_index(drop=True)

df_train_f = f_distancias.feature_distancias(df_train_f)
df_test_f = f_distancias.feature_distancias(df_test_f, df_train_f)

## GridSearchCV

In [5]:
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import GridSearchCV

features = ['antiguedad', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'lat', 'lng', 'fecha', 'idf_titulo', 'idf_descripcion',
       'peso_descripcion', 'porcentaje_metros', 'diferencia_metros',
       'metroscubiertos_bins_unif', 'metroscubiertos_bins_perc',
       'metros_totales_normalizados', 'metros_cubiertos_normalizados',
       'promedio_metros_tipo_propiedad', 'promedio_metros_cub_tipo_propiedad',
       'promedio_metros_cubiertos_provincia', 'mes', 'dia', 'dias_desde_datos',
       'meses_desde_datos', 'antiguedad_bins_perc', 'tam_ambientes',
       'promedio_precio_provincia', 'promedio_precio_ciudad',
       'promedio_precio_ciudad_gen', 'varianza_precio_ciudad', 'count_ciudad',
       'promedio_id_zona', 'promedio_id_zona_gen', 'varianza_id_zona',
       'count_id_zona', 'promedio_precio_tipo_propiedad',
       'promedio_precio_tipo_propiedad_ciudad',
       'promedio_precio_tipo_propiedad_ciudad_gen',
       'count_tipo_propiedad_ciudad', 'promedio_por_mes', 'varianza_por_mes',
       'promedio_precio_habitaciones_banos_garages',
       'promedio_precio_banos_garages', 'promedio_precio_hbg_tipo_propiedad',
       'lat_norm', 'lng_norm', 'puntaje', 'distancia_ciudad_centrica',
       'distancia_centro_mexico', 'distancia_ciudad_cara']

forest_params = {     
    'n_estimators': [10],
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_split': [2, 4, 8],
    "min_impurity_decrease": [0, 0.1, 0.2],
    'max_depth': [8, 12, 14],
    'n_jobs': [4]
}

x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=1)

gs_cv = GridSearchCV(estimator=RandomForestRegressor(), 
                           param_grid=forest_params, 
                           cv=4, 
                           verbose=3, scoring='neg_mean_absolute_error')

gs_cv.fit(x_train, y_train)

Fitting 4 folds for each of 81 candidates, totalling 324 fits
[CV] max_depth=8, max_features=auto, min_impurity_decrease=0, min_samples_split=2, n_estimators=10, n_jobs=4 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  max_depth=8, max_features=auto, min_impurity_decrease=0, min_samples_split=2, n_estimators=10, n_jobs=4, score=-608411.857, total=  12.6s
[CV] max_depth=8, max_features=auto, min_impurity_decrease=0, min_samples_split=2, n_estimators=10, n_jobs=4 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.6s remaining:    0.0s


[CV]  max_depth=8, max_features=auto, min_impurity_decrease=0, min_samples_split=2, n_estimators=10, n_jobs=4, score=-608258.219, total=  12.9s
[CV] max_depth=8, max_features=auto, min_impurity_decrease=0, min_samples_split=2, n_estimators=10, n_jobs=4 


...

[CV]  max_depth=14, max_features=sqrt, min_impurity_decrease=0, min_samples_split=4, n_estimators=10, n_jobs=4, score=-539969.209, total=   5.9s
[CV] max_depth=14, max_features=sqrt, min_impurity_decrease=0, min_samples_split=4, n_estimators=10, n_jobs=4 
[CV]  max_depth=14, max_features=sqrt, min_impurity_decrease=0, min_samples_split=4, n_estimators=10, n_jobs=4, score=-540163.801, total=   5.2s
[CV] max_depth=14, max_features=sqrt, min_impurity_decrease=0, min_samples_split=8, n_estimators=10, n_jobs=4 
[CV]  max_depth=14, max_features=sqrt, min_impurity_decrease=0, min_samples_split=8, n_estimators=10, n_jobs=4, score=-535053.635, total=   5.6s
[CV] max_depth=14, max_features=sqrt, min_impurity_decrease=0, min_samples_split=8, n_estimators=10, n_jobs=4 
[CV]  max_depth=14, max_features=sqrt, min_impurity_decrease=0, min_samples_split=8, n_estimators=10, n_jobs=4, score=-532086.242, total=   4.7s
[CV] max_depth=14, max_features=sqrt, min_impurity_decrease=0, min_samples_split=8, n_e

[CV]  max_depth=14, max_features=log2, min_impurity_decrease=0, min_samples_split=2, n_estimators=10, n_jobs=4, score=-542283.971, total=   4.3s
[CV] max_depth=14, max_features=log2, min_impurity_decrease=0, min_samples_split=2, n_estimators=10, n_jobs=4 
[CV]  max_depth=14, max_features=log2, min_impurity_decrease=0, min_samples_split=2, n_estimators=10, n_jobs=4, score=-543865.647, total=   5.4s
[CV] max_depth=14, max_features=log2, min_impurity_decrease=0, min_samples_split=4, n_estimators=10, n_jobs=4 
[CV]  max_depth=14, max_features=log2, min_impurity_decrease=0, min_samples_split=4, n_estimators=10, n_jobs=4, score=-545389.076, total=   4.3s
[CV] max_depth=14, max_features=log2, min_impurity_decrease=0, min_samples_split=4, n_estimators=10, n_jobs=4 
[CV]  max_depth=14, max_features=log2, min_impurity_decrease=0, min_samples_split=4, n_estimators=10, n_jobs=4, score=-540326.677, total=   4.6s
[CV] max_depth=14, max_features=log2, min_impurity_decrease=0, min_samples_split=4, n_e

[CV]  max_depth=14, max_features=log2, min_impurity_decrease=0.2, min_samples_split=8, n_estimators=10, n_jobs=4, score=-543547.355, total=   4.3s
[CV] max_depth=14, max_features=log2, min_impurity_decrease=0.2, min_samples_split=8, n_estimators=10, n_jobs=4 
[CV]  max_depth=14, max_features=log2, min_impurity_decrease=0.2, min_samples_split=8, n_estimators=10, n_jobs=4, score=-546395.718, total=   4.3s


[Parallel(n_jobs=1)]: Done 324 out of 324 | elapsed: 51.1min finished


GridSearchCV(cv=4, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [8, 12, 14],
     

In [10]:
display(gs_cv.best_params_)

{'max_depth': 14,
 'max_features': 'auto',
 'min_impurity_decrease': 0,
 'min_samples_split': 4,
 'n_estimators': 10,
 'n_jobs': 4}

In [11]:
pd.DataFrame(gs_cv.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,param_min_impurity_decrease,param_min_samples_split,param_n_estimators,param_n_jobs,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
55,23.260600,1.404574,0.236458,0.030572,14,auto,0,4,10,4,"{'max_depth': 14, 'max_features': 'auto', 'min...",-528349.121217,-530890.880154,-532082.478550,-535924.142561,-531811.630888,2730.513643,1
59,19.586377,1.409189,0.313867,0.073603,14,auto,0.1,8,10,4,"{'max_depth': 14, 'max_features': 'auto', 'min...",-530558.903369,-531093.159143,-532790.174399,-533335.518781,-531944.426302,1150.517018,2
62,16.355246,0.213069,0.223815,0.006928,14,auto,0.2,8,10,4,"{'max_depth': 14, 'max_features': 'auto', 'min...",-528378.215185,-529534.255552,-534137.417258,-535957.744650,-532001.873790,3139.645301,3
60,20.505637,2.488054,0.236575,0.034602,14,auto,0.2,2,10,4,"{'max_depth': 14, 'max_features': 'auto', 'min...",-529741.071354,-531333.423413,-531933.331044,-535211.104806,-532054.715529,1990.646430,4
57,18.102346,1.046656,0.219030,0.003049,14,auto,0.1,2,10,4,"{'max_depth': 14, 'max_features': 'auto', 'min...",-531462.813550,-529301.741433,-532805.094578,-535151.885867,-532180.363566,2122.591123,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22,5.019811,0.589344,0.314336,0.084504,8,log2,0.1,4,10,4,"{'max_depth': 8, 'max_features': 'log2', 'min_...",-628492.066397,-614291.338906,-627150.142178,-627503.102907,-624359.129109,5833.460220,77
20,3.190034,0.183011,0.235852,0.030457,8,log2,0,8,10,4,"{'max_depth': 8, 'max_features': 'log2', 'min_...",-629932.321955,-619255.655382,-624801.938138,-625534.881580,-624881.196023,3794.562144,78
26,4.076764,0.817855,0.227318,0.008544,8,log2,0.2,8,10,4,"{'max_depth': 8, 'max_features': 'log2', 'min_...",-631829.074128,-621866.734008,-626811.070221,-626565.487303,-626768.092316,3524.212881,79
25,3.947295,0.220190,0.251449,0.025081,8,log2,0.2,4,10,4,"{'max_depth': 8, 'max_features': 'log2', 'min_...",-626224.330154,-629238.630989,-623138.921975,-633220.884568,-627955.689391,3727.148718,80


In [12]:
display(gs_cv.best_params_)
from sklearn.externals import joblib
# Guarda el modelo
joblib.dump(gs_cv, 'randomforest_gscv_1.pkl', compress=1)

{'max_depth': 14,
 'max_features': 'auto',
 'min_impurity_decrease': 0,
 'min_samples_split': 4,
 'n_estimators': 10,
 'n_jobs': 4}

['randomforest_gscv_1.pkl']

In [30]:
from sklearn.ensemble import RandomForestRegressor


params = {'bootstrap': False,
 'max_features': 'sqrt',
 'min_samples_split': 4}

forest = RandomForestRegressor(verbose=1, n_estimators=100, **params)

features = ['habitaciones', 'garages','banos','antiguedad', 'metroscubiertos',  'metrostotales','lat_norm', 
           'lng_norm', 'gimnasio', 'usosmultiples', 'piscina']

features_test = ['top_provincia', 'promedio_precio_ciudad', 'anio', 'promedio_id_zona', 
                 'promedio_precio_tipo_propiedad', 'count_id_zona', 'count_ciudad', 'puntaje', 
               'count_tipo_propiedad_ciudad', 'promedio_precio_tipo_propiedad_ciudad_gen','count_id_zona'
           'dias_desde_datos','meses_desde_datos','porcentaje_metros','distancia_ciudad_centrica', 
                'distancia_centro_mexico', 'distancia_ciudad_cara']

features += features_test

x_train = utils.filtrar_features(df_train_f, features)
y_train = df_train_f['precio']
x_eval = utils.filtrar_features(df_test_f, features)
y_eval = df_test_f['precio']

forest.fit(x_train, y_train)

y_pred_train = forest.predict(x_train)
y_pred_eval = forest.predict(x_eval)

mae_train = utils.MAE(y_train, y_pred_train)
mae_eval = utils.MAE(y_eval, y_pred_eval)

print(f"MAE RandomForest (train): {mae_train:.5f}")
print(f"MAE RandomForest (eval): {mae_eval:.5f}")

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.6min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    9.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


MAE RandomForest (train): 58145.90947
MAE RandomForest (eval): 506702.25115


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    3.0s finished
